In [ ]:

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:


# Importing the Keras libraries and packages
# the required Libraries are imported.

import os
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.utils import class_weight, shuffle

from keras import applications
from keras import optimizers
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint










In [ ]:
foldernames = ['MOURNING DOVE', 'BROWN NOODY', 'DOWNY WOODPECKER', 'EMPEROR PENGUIN', 'WHITE TAILED TROPIC', 'OSPREY', 'HOUSE FINCH', 'AMERICAN COOT', 'GOLDEN PIPIT', 'PUFFIN']
categories = []
files = []
i = 0
flag=0

len(foldernames)

In [ ]:


for k, folder in enumerate(foldernames):
    filenames = os.listdir("../input/100-bird-species/train/" + folder);
    for file in filenames:
        
        files.append("../input/100-bird-species/train/" + folder + "/" + file)
       
        categories.append(k)
     

df = pd.DataFrame({
    'filename': files,
    'category': categories
})

Image and target name are separated.

In [ ]:
train_df = pd.DataFrame(columns=['filename', 'category'])
for i in range(70):
    train_df = train_df.append(df[df.category == i].iloc[:500,:])

train_df.head()
train_df = train_df.reset_index(drop=True)
train_df

In [ ]:
y = train_df['category']
x = train_df['filename']
x, y = shuffle(x, y, random_state=8)

I resized and centered images.

In [ ]:
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

images = []
for i, file_path in enumerate(train_df.filename.values):
        
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        img = centering_image(cv2.resize(img, dsize=tile_size))

        img = img[16:240, 16:240]
        images.append(img)
        

images = np.array(images)

Renaming Images.

In [ ]:
img_rows, img_cols, img_channel = 224, 224, 3
name_bird = []
for i in range(10):
    path = train_df[train_df.category == i].values[2]
    name_bird.append(path[0].split('/')[-2]) 

  

In [ ]:
  for a in range(10):
    print(a,name_bird[a])

Displaying the different categories of animals.

In [ ]:
rows,cols = 3,3
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(10,10))
for i in range(9):
    path = train_df[train_df.category == i].values[3]
    axes[i//cols, i%cols].set_title(name_bird[i])
    axes[i//cols, i%cols].imshow(images[train_df[train_df.filename == path[0]].index[0]])

I split the dataset into train and test.

In [ ]:
data_num = len(y)
random_index = np.random.permutation(data_num)

x_shuffle = []
y_shuffle = []
for i in range(data_num):
    x_shuffle.append(images[random_index[i]])
    y_shuffle.append(y[random_index[i]])
    
x = np.array(x_shuffle) 
y = np.array(y_shuffle)
val_split_num = int(round(0.2*len(y)))
x_train = x[val_split_num:]
y_train = y[val_split_num:]
x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

I created the classification model.

In [ ]:
modelName = "vgg16"
weights = '../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = applications.VGG16(include_top=False, weights=weights, input_shape=(img_rows, img_cols, img_channel))

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(10, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Training the model.

In [ ]:
batch_size = 32
epochs = 50

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)


history = model.fit(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc')]
)

Observing Accuracy of the Model.

In [ ]:
print("CNN: Epochs={0:d}, Train accuracy={1:.5f}, Validation accuracy={2:.5f}".format(epochs,history.history['accuracy'][epochs-1],history.history['val_accuracy'][epochs-1]))


Predictions for the model.

In [ ]:
test_images =[]
j = 30 
for i in range(6):
    path = train_df[train_df.category == i].values[j]
    a = images[train_df[train_df.filename == path[0]].index[0]]
    img = np.array(a)
    img = img[:, :, ::-1].copy() 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))
    img = centering_image(cv2.resize(img, dsize=tile_size))
    img = img[16:240, 16:240]
    test_images.append(img)

test_images = np.array(test_images).reshape(-1,224,224,3)
prediction = model.predict(test_images)
animals = name_bird
i = 0
for pred in prediction:
    path = train_df[train_df.category == i].values[2]
    plt.imshow(test_images[i])
    plt.show()
    print('Actual  :', animals[i])
    print('Predict :', animals[np.where(pred.max() == pred)[0][0]])
    i += 1